# AWS Machine Learning Nanodegree Capstone Project
# Data Importing and Transformation

## Summary

### In this notebook I will import the data, create a pipeline, explore the data, and train and test one or more models. I will then evaluate the results

In [1]:
#data import
import pandas as pd
import os
import glob

## Data import

### In this step I am importing data into Pandas dataframes, merging the various features into a single frame and resetting the index

In [10]:
data_path = os.path.join(os.getcwd(), 'use_this_data')

In [11]:
wl_path = os.path.join(data_path, 'watchlist_01012022_10312022')
rat_path = os.path.join(data_path, 'ratios_01012022_10312022')
sq_path = os.path.join(data_path, 'stockquotes_01012022_10312022')
ss_path = os.path.join(data_path, 'shortsales_01012022_10312022')

In [28]:
wl = pd.read_parquet(wl_path, columns=['Security Code Clean', 'file_date'])
pe = pd.read_parquet(rat_path, columns=['Security Code Clean', 'file_date', 'P/E ratio'])
pb = pd.read_parquet(rat_path, columns=['Security Code Clean', 'file_date', 'P/B ratio'])
trade_vol = pd.read_parquet(sq_path, columns=['Security Code Clean', 'file_date', 'Trade Volume'])
close_px = pd.read_parquet(sq_path, columns=['Security Code Clean', 'file_date', 'Closing Price'])
ss_vol = pd.read_parquet(ss_path, columns=['Security Code Clean', 'file_date', 'Trading Volume'])

In [13]:
#create pandas business date range datetimeindex:
#bd_range = pd.bdate_range(start='1/1/2022', end='10/31/2022')

In [29]:
#add indicator column for whether a stock is on the list
wl['on_watchlist'] = 1

In [30]:
wl.rename(columns={'Security Code Clean':'item_id','file_date':'timestamp','on_watchlist':'target_value'}, inplace=True)

In [25]:
#wl = wl.set_index('timestamp')
#reindex(bd_range, method=None)

In [31]:
wl

,item_id,timestamp,target_value
0,1213,2022-01-03,1
1,1418,2022-01-03,1
2,1472,2022-01-03,1
3,1512,2022-01-03,1
4,1538,2022-01-03,1
...,...,...,...
8,3043,2022-10-31,1
9,3536,2022-10-31,1
10,6225,2022-10-31,1
11,8101,2022-10-31,1


In [51]:
# Forecast requires timestamps with the following format: yyyy-MM-dd hh:mm:ss
wl['timestamp'] = wl['timestamp'].apply(lambda x: x.strftime("%Y-%m-%d %X"))

In [52]:
wl

,item_id,timestamp,target_value
0,1213,2022-01-03 00:00:00,1
1,1418,2022-01-03 00:00:00,1
2,1472,2022-01-03 00:00:00,1
3,1512,2022-01-03 00:00:00,1
4,1538,2022-01-03 00:00:00,1
...,...,...,...
8,3043,2022-10-31 00:00:00,1
9,3536,2022-10-31 00:00:00,1
10,6225,2022-10-31 00:00:00,1
11,8101,2022-10-31 00:00:00,1


In [5]:
close_px

,Security Code Clean,file_date,Closing Price
0,0050,2022-01-03,146.40
1,0051,2022-01-03,60.90
2,0052,2022-01-03,135.20
3,0053,2022-01-03,70.45
4,0054,2022-01-03,31.80
...,...,...,...
1171,9944,2022-10-31,20.40
1172,9945,2022-10-31,39.60
1173,9946,2022-10-31,11.55
1174,9955,2022-10-31,17.05


In [6]:
close_px.rename(columns={'Security Code Clean':'item_id', 
                           'Closing Price': 'target_value',
                            'file_date':'timestamp'}, inplace=True)

In [7]:
close_px

,item_id,timestamp,target_value
0,0050,2022-01-03,146.40
1,0051,2022-01-03,60.90
2,0052,2022-01-03,135.20
3,0053,2022-01-03,70.45
4,0054,2022-01-03,31.80
...,...,...,...
1171,9944,2022-10-31,20.40
1172,9945,2022-10-31,39.60
1173,9946,2022-10-31,11.55
1174,9955,2022-10-31,17.05


In [8]:
trade_vol

,Security Code Clean,file_date,Trade Volume
0,0050,2022-01-03,7064552
1,0051,2022-01-03,135357
2,0052,2022-01-03,882163
3,0053,2022-01-03,28349
4,0054,2022-01-03,3998
...,...,...,...
1171,9944,2022-10-31,71751
1172,9945,2022-10-31,6956898
1173,9946,2022-10-31,11298
1174,9955,2022-10-31,131905


In [9]:
trade_vol.rename(columns={'Security Code Clean':'item_id', 
                           'Trade Volume': 'target_value',
                            'file_date':'timestamp'}, inplace=True)

In [10]:
trade_vol

,item_id,timestamp,target_value
0,0050,2022-01-03,7064552
1,0051,2022-01-03,135357
2,0052,2022-01-03,882163
3,0053,2022-01-03,28349
4,0054,2022-01-03,3998
...,...,...,...
1171,9944,2022-10-31,71751
1172,9945,2022-10-31,6956898
1173,9946,2022-10-31,11298
1174,9955,2022-10-31,131905


In [ ]:
target_wl = wl.reindex(copy=True)

In [14]:
pe

,Security Code Clean,file_date,P/E ratio
0,1101,2022-01-03,13.91
1,1102,2022-01-03,10.09
2,1103,2022-01-03,7.29
3,1104,2022-01-03,12.78
4,1108,2022-01-03,20.52
...,...,...,...
960,9944,2022-10-31,4.11
961,9945,2022-10-31,5.34
962,9946,2022-10-31,18.33
963,9955,2022-10-31,NaN


In [15]:
pe.rename(columns={'Security Code Clean':'item_id', 'P/E ratio': 'target_value', 'file_date':'timestamp'},inplace=True)

In [16]:
pe

,item_id,timestamp,target_value
0,1101,2022-01-03,13.91
1,1102,2022-01-03,10.09
2,1103,2022-01-03,7.29
3,1104,2022-01-03,12.78
4,1108,2022-01-03,20.52
...,...,...,...
960,9944,2022-10-31,4.11
961,9945,2022-10-31,5.34
962,9946,2022-10-31,18.33
963,9955,2022-10-31,NaN


In [19]:
ss_vol.rename(columns={'Security Code Clean':'item_id',
                   'Trading Volume':'ss_trading_vol',
                   'Trading Value':'ss_trading_value'}, inplace=True)

In [20]:
ss_vol

,sec_code,file_date,ss_trading_vol
0,0050,2022-01-03,2
1,0051,2022-01-03,0
2,0052,2022-01-03,0
3,0053,2022-01-03,0
4,0054,2022-01-03,0
...,...,...,...
1118,9944,2022-10-31,0
1119,9945,2022-10-31,173
1120,9946,2022-10-31,0
1121,9955,2022-10-31,0


In [18]:
all1 = pd.merge(left=sq, right=ss, how='left', on=['sec_code', 'file_date'], suffixes=('_sq','_ss'))

In [19]:
all2 = pd.merge(left=all1, right=rat, how='left', on=['sec_code', 'file_date'], suffixes=('_sq_ss', '_rat'))

In [26]:
all = pd.merge(left=all2, right=wl, how='left', on=['sec_code', 'file_date'], suffixes=('_sq_ss_rat', '_wl'))

In [27]:
#showing watchlist rows
all[all['on_watchlist']==1]

,sec_code,file_date,trade_volume,closing_price,change,ss_trading_vol,ss_trading_value,pe_ratio,pb_ratio,on_watchlist
171,1213,2022-01-03,2001,8.42,0.00,0,0,NaN,1.04,1.0
221,1418,2022-01-03,71316,12.70,1.05,0,0,NaN,3.80,1.0
257,1472,2022-01-03,2,NaN,0.00,0,0,NaN,3.42,1.0
267,1512,2022-01-03,354293,4.41,0.40,0,0,NaN,12.60,1.0
290,1538,2022-01-03,59844,10.40,0.15,0,0,NaN,2.36,1.0
...,...,...,...,...,...,...,...,...,...,...
234859,3043,2022-10-31,204329,6.42,0.10,0,0,22.93,5.14,1.0
234922,3536,2022-10-31,6000,4.15,0.05,0,0,NaN,7.03,1.0
235111,6225,2022-10-31,39220,6.30,0.12,0,0,NaN,27.39,1.0
235207,8101,2022-10-31,75749,5.90,0.00,0,0,3.88,2.71,1.0


In [28]:
all

,sec_code,file_date,trade_volume,closing_price,change,ss_trading_vol,ss_trading_value,pe_ratio,pb_ratio,on_watchlist
0,0050,2022-01-03,7064552,146.40,0.90,2,292950,NaN,NaN,NaN
1,0051,2022-01-03,135357,60.90,0.10,0,0,NaN,NaN,NaN
2,0052,2022-01-03,882163,135.20,1.05,0,0,NaN,NaN,NaN
3,0053,2022-01-03,28349,70.45,0.50,0,0,NaN,NaN,NaN
4,0054,2022-01-03,3998,31.80,0.12,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
235296,9944,2022-10-31,71751,20.40,0.05,0,0,4.11,0.66,NaN
235297,9945,2022-10-31,6956898,39.60,0.40,173,6891050,5.34,2.96,NaN
235298,9946,2022-10-31,11298,11.55,0.05,0,0,18.33,0.59,NaN
235299,9955,2022-10-31,131905,17.05,0.25,0,0,NaN,1.21,NaN


In [29]:
#Investigate Nan - we can see multiple Nans indicating that not every stock has a value for all columns
all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 235301 entries, 0 to 235300
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   sec_code          235301 non-null  object        
 1   file_date         235301 non-null  datetime64[ns]
 2   trade_volume      235301 non-null  Int64         
 3   closing_price     232340 non-null  float64       
 4   change            235301 non-null  float64       
 5   ss_trading_vol    224945 non-null  Int64         
 6   ss_trading_value  224945 non-null  Int64         
 7   pe_ratio          168174 non-null  float64       
 8   pb_ratio          194346 non-null  float64       
 9   on_watchlist      3440 non-null    float64       
dtypes: Int64(3), datetime64[ns](1), float64(5), object(1)
memory usage: 20.4+ MB


In [30]:
#count of nulls per column
print(all.isnull().sum())

sec_code                 0
file_date                0
trade_volume             0
closing_price         2961
change                   0
ss_trading_vol       10356
ss_trading_value     10356
pe_ratio             67127
pb_ratio             40955
on_watchlist        231861
dtype: int64


## Export data to prepare for Amazon Forecast

In [53]:
#setting index=False because Forecast cannot receive any extra attributes during import
wl.to_parquet('./forecast_import/target_wl.parquet',index=False)